In [ ]:
# Code to mount the drive 
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#For Himanshu's access
import os
path = "/content/drive/MyDrive/CollabProjects/685/catr"
os.chdir(path)

In [ ]:
#For people with shared access
%cd '/content/drive/MyDrive/685/catr/'

[Errno 2] No such file or directory: '/content/drive/MyDrive/685/catr/'
/content


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 3.3 MB 8.1 MB/s 
     |████████████████████████████████| 3.3 MB 63.5 MB/s 
     |████████████████████████████████| 596 kB 82.9 MB/s 
     |████████████████████████████████| 895 kB 81.0 MB/s 
     |████████████████████████████████| 61 kB 599 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch

from transformers import BertTokenizer,BertForMaskedLM

from PIL import Image
import argparse

from models import caption
from datasets import coco, utils
from configuration import Config
import os

In [ ]:
# Assign image folder, image path and mmodel checkpoint path 
image_path = '../sarcasm/test/'
img_name   = '82ca5176fe4e7e74d70e1c7bb7988e9.jpg'
img_final_loc = image_path+img_name
checkpoint_path = './checkpoint_sarcasm_1.pth'

In [ ]:
# Create model configurations 
config = Config()
# Check if the model is able to find the checkpoint
if not os.path.exists(checkpoint_path):
  raise NotImplementedError('Give valid checkpoint path')

In [ ]:
# Load the finetuned model in our case
model,_ = caption.build_model(config)

In [ ]:
# Map location to CPu to unserialize the model using CPU 
checkpoint = torch.load(checkpoint_path, map_location='cpu')
# Load model dictionary 
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [ ]:
# Load BERT tokenizer
tokenizer   = BertTokenizer.from_pretrained('bert-base-uncased')
start_token = tokenizer.convert_tokens_to_ids(tokenizer._cls_token)
end_token = tokenizer.convert_tokens_to_ids(tokenizer._sep_token)

In [ ]:
# Load image and perform pre-processing 
image = Image.open(img_final_loc)
image = coco.val_transform(image)
image = image.unsqueeze(0)

In [ ]:
def create_caption_and_mask(start_token, max_length):
    caption_template = torch.zeros((1, max_length), dtype=torch.long)
    mask_template = torch.ones((1, max_length), dtype=torch.bool)

    caption_template[:, 0] = start_token
    mask_template[:, 0] = False

    return caption_template, mask_template

In [ ]:
caption, cap_mask = create_caption_and_mask(
    start_token, config.max_position_embeddings)

In [ ]:
@torch.no_grad()
def evaluate():
    model.eval()
    for i in range(config.max_position_embeddings - 1):
        predictions = model(image, caption, cap_mask)
        predictions = predictions[:, i, :]
        predicted_id = torch.argmax(predictions, axis=-1)

        if predicted_id[0] == 102:
            return caption

        caption[:, i+1] = predicted_id[0]
        cap_mask[:, i+1] = False

    return caption

In [ ]:
output = evaluate()

/content/drive/My Drive/CollabProjects/685/catr/models/position_encoding.py:38: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


In [ ]:
result = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
result

"i'm sure they think she's doing."

In [ ]:
# GPT-2 for calculation of the requisite perplexity 
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
device = 'cpu'
model_id = 'gpt2-large'
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
model.eval()

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [ ]:
from tqdm import tqdm
import torch

In [ ]:
''.join("I love sunny weather")

'I love sunny weather'

In [ ]:
encodings = tokenizer(''.join('I love sunny weather and he loves it too. But fokmdsfknr hjd reason he hates it when thklnsdings do not go as planned.'), return_tensors='pt')
encodings

{'input_ids': tensor([[   40,  1842, 27737,  6193,   290,   339, 10408,   340,  1165,    13,
           887,   277,   482,  9132, 28202, 15418,    81,   289,    73,    67,
          1738,   339, 24702,   340,   618,   294, 41582,  5907,    67,   654,
           466,   407,   467,   355,  6027,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# Links used for calculation: https://huggingface.co/docs/transformers/perplexity
# Link 2: https://discuss.huggingface.co/t/guide-the-best-way-to-calculate-the-perplexity-of-fixed-length-models/193/2
# Perplexity calculation function
# Bleu Score won't be applicable most probably 
max_length = model.config.n_positions
stride = 3

lls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i    # may be different from stride on last loop
    input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:,:-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        log_likelihood = outputs[0] * trg_len

    lls.append(log_likelihood)

ppl = torch.exp(torch.stack(lls).sum() / end_loc)

100%|██████████| 12/12 [00:12<00:00,  1.01s/it]


In [ ]:
#

tensor(187.4026)

In [ ]:
# Perplexity Calculation for BERT model
# Should the perplexity calculation be done by model or BERT, my guess is BERT should suffice
with torch.no_grad():
    bertMaskedLM = BertForMaskedLM.from_pretrained('bert-base-uncased')
    bert_model.eval() #Required for deterministic output as dropouts may affect 

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=bertMaskedLM(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(),tensor_input.squeeze()).data 
    return math.exp(loss)

In [ ]:
import numpy as np
for j in range(1):
    print(j, [get_score(i) for i in np_array[j]])

ImportError: ignored